In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("Csv-Üzeri-SQL") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

## Veriyi okurken sadece tarih-saat sütununu seçelim

In [2]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("D:\\Datasets\\OnlineRetail.csv") \
.select("InvoiceDate").distinct()

In [3]:
df.show(5)

+----------------+
|     InvoiceDate|
+----------------+
| 3.12.2010 16:50|
| 7.12.2010 12:28|
| 8.12.2010 15:02|
|10.12.2010 09:53|
|12.12.2010 13:32|
+----------------+
only showing top 5 rows



Formatı anlamak için beş satırdan gün ve ay belli olmuyor daha fazla satır görelim.

In [4]:
df.show(15)

+----------------+
|     InvoiceDate|
+----------------+
| 3.12.2010 16:50|
| 7.12.2010 12:28|
| 8.12.2010 15:02|
|10.12.2010 09:53|
|12.12.2010 13:32|
|15.12.2010 13:21|
|16.12.2010 08:41|
|17.12.2010 09:52|
| 9.01.2011 11:43|
|11.01.2011 11:38|
|16.01.2011 15:50|
|25.01.2011 17:06|
|27.01.2011 12:10|
|28.01.2011 12:19|
|31.01.2011 12:16|
+----------------+
only showing top 15 rows



Evet şimdi anlaşıldı. Format gün.ay.yıl saat:dakika
yani dd.MM.yyyy HH:mm

Datetime da ise varsayılan format yyyy-MM-dd HH:mm:ss

In [5]:
mevcut_format = 'dd.MM.yyyy HH:mm'

# Örnek TarihSaat Operasyonları

In [6]:
from pyspark.sql import functions as F

df2 = df.withColumn("InvoiceDate", F.trim(F.col("InvoiceDate"))) \
.withColumn("normal_tarih", F.to_date(F.col("InvoiceDate"), mevcut_format)) \
.withColumn("standart_ts", F.to_timestamp(F.col("InvoiceDate"), mevcut_format)) \

df2.show(10)

+----------------+------------+-------------------+
|     InvoiceDate|normal_tarih|        standart_ts|
+----------------+------------+-------------------+
| 3.12.2010 16:50|  2010-12-03|2010-12-03 16:50:00|
| 7.12.2010 12:28|  2010-12-07|2010-12-07 12:28:00|
| 8.12.2010 15:02|  2010-12-08|2010-12-08 15:02:00|
|10.12.2010 09:53|  2010-12-10|2010-12-10 09:53:00|
|12.12.2010 13:32|  2010-12-12|2010-12-12 13:32:00|
|15.12.2010 13:21|  2010-12-15|2010-12-15 13:21:00|
|16.12.2010 08:41|  2010-12-16|2010-12-16 08:41:00|
|17.12.2010 09:52|  2010-12-17|2010-12-17 09:52:00|
| 9.01.2011 11:43|  2011-01-09|2011-01-09 11:43:00|
|11.01.2011 11:38|  2011-01-11|2011-01-11 11:38:00|
+----------------+------------+-------------------+
only showing top 10 rows



# Tarih Formatı Değiştirme

In [7]:
format_tr = "dd/MM/yyyy HH:mm:ss"
format_eng = "MM-dd-yyyy HH:mm:ss"

df3 = df2 \
.withColumn("TSTR", F.date_format(F.col("standart_ts"), format_tr)) \
.withColumn("TSENG", F.date_format(F.col("standart_ts"), format_eng)) \
.withColumn("unix_time", F.unix_timestamp(F.col("standart_ts"))) \

df3.show(10)

+----------------+------------+-------------------+-------------------+-------------------+----------+
|     InvoiceDate|normal_tarih|        standart_ts|               TSTR|              TSENG| unix_time|
+----------------+------------+-------------------+-------------------+-------------------+----------+
| 3.12.2010 16:50|  2010-12-03|2010-12-03 16:50:00|03/12/2010 16:50:00|12-03-2010 16:50:00|1291384200|
| 7.12.2010 12:28|  2010-12-07|2010-12-07 12:28:00|07/12/2010 12:28:00|12-07-2010 12:28:00|1291714080|
| 8.12.2010 15:02|  2010-12-08|2010-12-08 15:02:00|08/12/2010 15:02:00|12-08-2010 15:02:00|1291809720|
|10.12.2010 09:53|  2010-12-10|2010-12-10 09:53:00|10/12/2010 09:53:00|12-10-2010 09:53:00|1291963980|
|12.12.2010 13:32|  2010-12-12|2010-12-12 13:32:00|12/12/2010 13:32:00|12-12-2010 13:32:00|1292149920|
|15.12.2010 13:21|  2010-12-15|2010-12-15 13:21:00|15/12/2010 13:21:00|12-15-2010 13:21:00|1292408460|
|16.12.2010 08:41|  2010-12-16|2010-12-16 08:41:00|16/12/2010 08:41:00|12

# Tarih ekleme, tarih farkı, timestamp içinden yılı alma

In [8]:
df4 = df2 \
.withColumn("bir_yil", F.date_add(F.col("standart_ts"), 365)) \
.withColumn("yil", F.year(F.col("standart_ts"))) \
.withColumn("fark", F.datediff(F.col("bir_yil"), F.col("standart_ts")))


df4.show(10)

+----------------+------------+-------------------+----------+----+----+
|     InvoiceDate|normal_tarih|        standart_ts|   bir_yil| yil|fark|
+----------------+------------+-------------------+----------+----+----+
| 3.12.2010 16:50|  2010-12-03|2010-12-03 16:50:00|2011-12-03|2010| 365|
| 7.12.2010 12:28|  2010-12-07|2010-12-07 12:28:00|2011-12-07|2010| 365|
| 8.12.2010 15:02|  2010-12-08|2010-12-08 15:02:00|2011-12-08|2010| 365|
|10.12.2010 09:53|  2010-12-10|2010-12-10 09:53:00|2011-12-10|2010| 365|
|12.12.2010 13:32|  2010-12-12|2010-12-12 13:32:00|2011-12-12|2010| 365|
|15.12.2010 13:21|  2010-12-15|2010-12-15 13:21:00|2011-12-15|2010| 365|
|16.12.2010 08:41|  2010-12-16|2010-12-16 08:41:00|2011-12-16|2010| 365|
|17.12.2010 09:52|  2010-12-17|2010-12-17 09:52:00|2011-12-17|2010| 365|
| 9.01.2011 11:43|  2011-01-09|2011-01-09 11:43:00|2012-01-09|2011| 365|
|11.01.2011 11:38|  2011-01-11|2011-01-11 11:38:00|2012-01-11|2011| 365|
+----------------+------------+-------------------+